In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (15, 5)
# Load a sort of library file with csv column names, shared with script that preps csv files for import
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import prepcsv.lib.prepcsv as prepcsv
csv_columns = prepcsv.file_columns
print(csv_columns.keys())

df = {}
for filename, column_names in csv_columns.items():
    cols = [c for c in csv_columns[filename] if c != 'url']
    df[filename] = pd.read_csv(f'./f1db_csv/{filename}.csv', usecols=cols)

In [ ]:
# Merge dataset to get all the data in a couple of usefull (but big) dataframes

def join_datasets(left, right, on):
    return df[left].join(df[right].set_index(on), on=on, rsuffix=f'_{right}')
df['RaCi']         = join_datasets('races', 'circuits', 'circuitId')
df['ReDr']         = join_datasets('results', 'driver', 'driverId') 
df['ReDrCo']       = join_datasets('ReDr', 'constructors', 'constructorId')
df['ReDrCoRaCi']   = join_datasets('ReDrCo', 'RaCi', 'raceId')
df['ReDrCoRaCiSt'] = join_datasets('ReDrCoRaCi', 'status', 'statusId')
df['StDr']         = join_datasets('driver_standings', 'driver', 'driverId')
df['StDrRa']       = join_datasets('StDr', 'races', 'raceId')
big_results = df['ReDrCoRaCiSt']
big_results

## Print how many victorys, and the list of victories

In [ ]:
def wins_of_driver(driver):
    res = big_results[(big_results['driverRef'] == driver) & (big_results['position'] == '1')]
    print(len(res))
    print(res[['year','circuitName']])
wins_of_driver('max_verstappen')

## Plot the number of races per season

In [ ]:
season_lengths = { year : len(df['races'][df['races']['year'] == year]) for year in range(1950, 2021)}
ts = pd.Series(season_lengths)
p = ts.plot(kind='bar', figsize=(20,5))
plt.show()

## Number of race wins per driver

In [ ]:
# winning_drivers = df_results_drivers_races_constructors[df_results_drivers['position'] == '1']
winning_drivers = big_results[big_results['position'] == '1']
win_counts = winning_drivers['driverRef'].value_counts()
win_counts[:20].plot(kind="bar", figsize=(20,10))
plt.show()
win_counts[:20]

In [ ]:
def when_champion(year):
    season_length = len(df['races'][df['races']['year'] == year])
    standings = df['StDrRa']
    season_races = pd.Series(df['ReDrCoRaCi'][df['ReDrCoRaCi']['year'] == year]['raceId'])
    season_standings = standings[standings['raceId'].isin(season_races)]
    points_for_winning = df['ReDrCoRaCi'][df['ReDrCoRaCi']['raceId'].isin(season_races)]['points'].max()
    for roundNumber in range(1,season_length+1):
        r = season_standings[season_standings['round'] == roundNumber]
        leader = r[r['position'] == 1]
        runnerup = r[r['position'] == 2]
        rounds_left = season_length - roundNumber
        if (rounds_left * points_for_winning) < leader.iloc[0]['points'] - runnerup.iloc[0]['points']:
            return (roundNumber, season_length)
when_champion(2016)

## List every seasons champion

In [ ]:
season_lengths = { year : len(df['races'][df['races']['year'] == year]) for year in range(1950, 2021)}
standings = df['StDrRa']
champs = {}
for year in range(1950, 2019):
    season_races = pd.Series(df['races'][df['races']['year'] == year]['raceId'])
    season_standings = standings[standings['raceId'].isin(season_races)]
    last_round = season_standings[season_standings['round'] == season_lengths[year]]
    champion = last_round[last_round['position'] == 1]
    when = when_champion(year)
    percentage = when[0]/(when[1]/100)
    champs[year] =  champion.iloc[0]['driverRef']
    print('{}: {} {} {} {:.1f}'.format(champion.iloc[0]['year'], 
                                champion.iloc[0]['forename'], 
                                champion.iloc[0]['surname'],
                                when,
                                percentage))

## Plot season lenght vs when the champion was known

In [ ]:
season_lengths = { year : len(df['races'][df['races']['year'] == year]) for year in range(1950, 2021)}
season_stacked = {}
for year in range(1950, 2018):
    season_stacked[year] = when_champion(year)

rows = zip(season_lengths.keys(), 
           [v[0] for v in season_stacked.values()],
           [v[1] for v in season_stacked.values()])
headers = ['year', 'champion_known', 'season_length']
stacked = pd.DataFrame(rows, columns=headers).set_index('year')[20:]

stacked.plot(kind='bar', figsize=(20,10))

## Dnf percentage per driver

In [ ]:
br = big_results
# print(br.columns)
twenty_nineteen_drivers = br[(br['year'] == 2019) & (br['round'] == 2)]
driver_stats = {}
for i in range(len(twenty_nineteen_drivers)):
    driverId = twenty_nineteen_drivers.iloc[i]['driverId']
    all_races = br[br.driverId == driverId]
    finished = all_races[(all_races.status.str.contains('^\+')) | (all_races.status.str.contains('Finished'))]
    percentage = len(finished)/(len(all_races)/100)
    print('{} - {} {}: {:.1f}%'.format(twenty_nineteen_drivers.iloc[i]['teamName'],
                                  twenty_nineteen_drivers.iloc[i]['forename'],
                                  twenty_nineteen_drivers.iloc[i]['surname'],
                                  percentage))
    driver_stats[twenty_nineteen_drivers.iloc[i]['driverRef']] = percentage
pd.Series(driver_stats).plot(kind='bar')    

## Circuits a driver has never won

In [ ]:
won = br[(br['driverRef'] == 'hamilton') & (br['position'] == '1')]
circuits_won = set([race for race in won['circuitName']])
races_2019 = df['RaCi'][df['RaCi']['year'] == 2019]
for circuit in races_2019['circuitName']:
    if circuit not in circuits_won:
        print(circuit)

## Lap times over the years per circuit (in progress)

In [ ]:
qualy = df['qualifying']
fastest_laps = qualy[qualy['position'] == 1]
fastest_laps[:20]

## Print the circuits a driver has won and the years he has done it

In [ ]:
driverRef = 'hamilton'
winning_races = br[(br['position'] == '1') & (br['driverRef'] == driverRef)]
winning_years = {}
for index, race in winning_races.iterrows():
    r = ''
    try:
        if champs[race['year']] == driverRef:
            r = str(race['year']) + '*'
        else:
            r = str(race['year']) + ' '
    except KeyError:
        continue
    try:
        winning_years[race['circuitName']].append(r)
    except KeyError:
        winning_years[race['circuitName']] = [r]
sorted_keys = sorted(winning_years, key=lambda k: len(winning_years[k]), reverse=True)
for race in sorted_keys:
    padding = 26 - len(race)
    print('{}:{}{}'.format(race, ' '*padding, ', '.join(winning_years[race]))) 

## Same as above, different representation

In [ ]:
driverRef = 'hamilton'
winning_races = br[(br['position'] == '1') & (br['driverRef'] == driverRef)]
winning_years = {}
for index, race in winning_races.iterrows():
    r = ''
    try:
        if champs[race['year']] == driverRef:
            r = str(race['year']) + 'C'
        else:
            r = str(race['year']) + ' '
    except KeyError:
        r = str(race['year']) + ' ' 
    try:
        winning_years[race['circuitName']].append(r)
    except KeyError:
        winning_years[race['circuitName']] = [r]
from more_itertools import flatten
years = sorted(set(flatten(winning_years.values())))
print('{} | {}  (C=champion)'.format(26*' ', '| '.join([y[2:] for y in years])))
sorted_keys = sorted(winning_years) #, key=lambda k: len(winning_years[k]), reverse=True)
for race in sorted_keys:
    padding = 26 - len(race)
    s = '{}:{}'.format(race, ' '*padding)
    for y in years:
        if y in winning_years[race]:
            s += '| *  '
        else:
            s += '|    '    
    print(s)